# Teste quantificador do efeito de Tranfered Learning, Data Augmentation, Pre-Processamento e combinações destes na classificação de HIC

K = 10 </br>
Resnet50 </br>
baseline </br>
baseline com transfered learning </br>
baseline com data aug </br>
baseline com transfered learning e data aug </br>

## Imports e Installs

In [ ]:
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout, Rescaling
from tensorflow.keras import optimizers
from tensorflow.keras.utils import image_dataset_from_directory

!pip install pydicom
import numpy as np
import pydicom
from PIL import Image
import os
import shutil

import cv2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


## Data Set

### Baixando

In [ ]:
!kaggle datasets download -d pedroamaro/rsna-ds-w-35k-files-k-folders

100% 7.24G/7.25G [01:02<00:00, 176MB/s]
100% 7.25G/7.25G [01:02<00:00, 124MB/s]


In [ ]:
!unzip /content/rsna-ds-w-35k-files-k-folders.zip | awk 'BEGIN {ORS=" "} {if(NR%1000==0)print "."}'
!rm /content/rsna-ds-w-35k-files-k-folders.zip

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

In [ ]:
total_yes = 0
total_no = 0

for i in range(10):
    yesdir = "/content/k"+str(i)+"/yes/"
    nodir = "/content/k"+str(i)+"/no/"
    _, _, files = next(os.walk(yesdir))
    total_yes += len(files)
    _, _, files = next(os.walk(nodir))
    total_no += len(files)

print("Total de Imagens NO:")
print(total_no)
print("Total de Imagens YES:")
print(total_yes)

Total de Imagens NO:
30069
Total de Imagens YES:
4930


### Salvando como jpg

In [ ]:
!mkdir jpg_k0
!mkdir jpg_k1
!mkdir jpg_k2
!mkdir jpg_k3
!mkdir jpg_k4
!mkdir jpg_k5
!mkdir jpg_k6
!mkdir jpg_k7
!mkdir jpg_k8
!mkdir jpg_k9

!mkdir jpg_k0/yes
!mkdir jpg_k1/yes
!mkdir jpg_k2/yes
!mkdir jpg_k3/yes
!mkdir jpg_k4/yes
!mkdir jpg_k5/yes
!mkdir jpg_k6/yes
!mkdir jpg_k7/yes
!mkdir jpg_k8/yes
!mkdir jpg_k9/yes

!mkdir jpg_k0/no
!mkdir jpg_k1/no
!mkdir jpg_k2/no
!mkdir jpg_k3/no
!mkdir jpg_k4/no
!mkdir jpg_k5/no
!mkdir jpg_k6/no
!mkdir jpg_k7/no
!mkdir jpg_k8/no
!mkdir jpg_k9/no

In [ ]:
def save_W_as_jpg(path, id, dest_folder):

  dcm = pydicom.dcmread(path)

  im = dcm.pixel_array.astype("float32") * dcm.RescaleSlope + dcm.RescaleIntercept

  rescaled_im = (np.maximum(im, 0)/im.max())*255.0
  final_im = np.uint8(rescaled_im)

  final_im = Image.fromarray(final_im)
  final_im.save(dest_folder+id+'.jpg')

In [ ]:
for i in range(10):
    dir = "/content/k"+str(i)+"/yes/"
    folder = os.listdir(dir)
    dest_folder = "/content/jpg_k"+str(i)+"/yes/"

    for id_ in folder:
      img_id = id_[:12]
      save_W_as_jpg(dir + f"{id_}", img_id, dest_folder)
      os.remove(dir + f"{id_}")

In [ ]:
for i in range(10):
    dir = "/content/k"+str(i)+"/no/"
    folder = os.listdir(dir)
    dest_folder = "/content/jpg_k"+str(i)+"/no/"

    for id_ in folder:
      img_id = id_[:12]
      save_W_as_jpg(dir + f"{id_}", img_id, dest_folder)
      os.remove(dir + f"{id_}")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


In [ ]:
total_yes = 0
total_no = 0

for i in range(10):
    yesdir = "/content/jpg_k"+str(i)+"/yes/"
    nodir = "/content/jpg_k"+str(i)+"/no/"
    _, _, files = next(os.walk(yesdir))
    total_yes += len(files)
    _, _, files = next(os.walk(nodir))
    total_no += len(files)

print("Total de Imagens JPG EM NO:")
print(total_no)
print("Total de Imagens JPG EM YES:")
print(total_yes)

Total de Imagens JPG EM NO:
30069
Total de Imagens JPG EM YES:
4930


In [ ]:
!rmdir k0/yes
!rmdir k1/yes
!rmdir k2/yes
!rmdir k3/yes
!rmdir k4/yes
!rmdir k5/yes
!rmdir k6/yes
!rmdir k7/yes
!rmdir k8/yes
!rmdir k9/yes

!rmdir k0/no
!rmdir k1/no
!rmdir k2/no
!rmdir k3/no
!rmdir k4/no
!rmdir k5/no
!rmdir k6/no
!rmdir k7/no
!rmdir k8/no
!rmdir k9/no

!rmdir k0
!rmdir k1
!rmdir k2
!rmdir k3
!rmdir k4
!rmdir k5
!rmdir k6
!rmdir k7
!rmdir k8
!rmdir k9

## Experimento

In [ ]:
!mkdir train
!mkdir train/yes
!mkdir train/no

### [ok] Baseline: WO/ Tranfered Learning

#### Declarações

In [ ]:
def clear_train_dir():
  for file in os.listdir("/content/train/yes/"):
    os.remove("/content/train/yes/" + file)

  for file in os.listdir("/content/train/no/"):
    os.remove("/content/train/no/" + file)

In [ ]:
def load_train_dir(ki):
  k = [0,1,2,3,4,5,6,7,8,9]
  for i in range(10):
    if i!= k[ki]:
      yesdir = "/content/jpg_k"+str(i)+"/yes/"
      nodir = "/content/jpg_k"+str(i)+"/no/"
      for file in os.listdir(yesdir):
        shutil.copy(yesdir + file, "/content/train/yes")
      for file in os.listdir(nodir):
        shutil.copy(nodir + file, "/content/train/no")

In [ ]:
def train_model(w0, w1):
    train_data = image_dataset_from_directory(
      ("/content/train"),
      labels='inferred',
      validation_split=1/3,
      subset="training",
      seed=123,
      batch_size = 64,
      color_mode = "rgb"
    )
  
    val_data = image_dataset_from_directory(
      "/content/train",
      labels='inferred',
      validation_split=1/3,
      subset="validation",
      seed=123,
      batch_size = 64,
      color_mode = "rgb"
    )

    class_names = train_data.class_names

    normalization_layer = Rescaling(1./255)

    normalized_data = train_data.map(lambda x, y: (normalization_layer(x), y))
    image_batch, labels_batch = next(iter(normalized_data))

    normalized_val_data = val_data.map(lambda x, y: (normalization_layer(x), y))
    val_image_batch, val_labels_batch = next(iter(normalized_val_data))

    keras.backend.clear_session()

    resnet_model = Sequential()

    resnet = tf.keras.applications.resnet50.ResNet50(
        include_top=False,
        input_shape=image_batch.shape[1:],
        pooling='avg',
        weights=None,
        classes=2,
    )

    resnet_model.add(resnet)

    resnet_model.add(Flatten())

    resnet_model.add(Dense(512, activation="relu"))

    resnet_model.add(Dropout(0.5))

    resnet_model.add(Dense(1, activation="sigmoid"))

    resnet_model.compile(loss='binary_crossentropy', 
                         optimizer=optimizers.Adam(learning_rate=0.00001),
                         metrics=[keras.metrics.AUC(), 'accuracy'])
  
    run_hist = resnet_model.fit(
      normalized_data, 
      validation_data=normalized_val_data, epochs=20, class_weight={0:w0, 1:w1}, 
      callbacks=[keras.callbacks.EarlyStopping(
                      monitor="val_auc",
                      min_delta=0,
                      patience=5,
                      verbose=1,
                      mode="auto",
                      baseline=None,
                      restore_best_weights=True,
                  )]
    )

    return resnet_model

In [ ]:
def test_model(resnet_model, test_path):
    test_data = image_dataset_from_directory(
      test_path,
      labels='inferred',
      label_mode="int",
      batch_size = 64,
      color_mode = "rgb"
    )

    normalization_layer = Rescaling(1./255)

    normalized_test_data = test_data.map(lambda x, y: (normalization_layer(x), y))
    test_image_batch, test_labels_batch = next(iter(normalized_test_data))

    results = resnet_model.evaluate(normalized_test_data)
    return results

In [ ]:
neg = 27062
pos = 4437
total = pos+neg

w0 = (1 / neg) * (total / 2.0)
w1 = (1 / pos) * (total / 2.0)

print('Peso para a classe 0: {:.2f}'.format(w0))
print('Peso para a classe 1: {:.2f}'.format(w1))

Peso para a classe 0: 0.58
Peso para a classe 1: 3.55


In [ ]:
print('Numero de imagens: NO')
!ls /content/train/no | wc -l
print('Numero de imagens: YES')
!ls /content/train/yes | wc -l

Numero de imagens: NO
0
Numero de imagens: YES
0


#### k0

In [ ]:
i = 0
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i))) #tava errado antes, por isso o erro na saida. Concertado na proxima celula

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/20
329/329 [==============================] - 313s 884ms/step - loss: 0.7394 - auc: 0.6303 - accuracy: 0.5859 - val_loss: 0.5286 - val_auc: 0.5958 - val_accuracy: 0.8591
Epoch 2/20
329/329 [==============================] - 292s 886ms/step - loss: 0.6066 - auc: 0.7290 - accuracy: 0.6268 - val_loss: 1.6802 - val_auc: 0.5246 - val_accuracy: 0.1844
Epoch 3/20
329/329 [==============================] - 293s 890ms/step - loss: 0.5606 - auc: 0.7715 - accuracy: 0.6415 - val_loss: 0.5025 - val_auc: 0.7743 - val_accuracy: 0.7332
Epoch 4/20
329/329 [==============================] - 292s 887ms/step - loss: 0.5354 - auc: 0.7976 - accuracy: 0.6680 - val_loss: 0.5521 - val_auc: 0.7921 - val_accuracy: 0.6624
Epoch 5/20
329/329 [==============================] - 293s 888ms/step - loss: 0.4975 - auc: 0.8314 - accuracy: 0.7064 - val_loss: 0.5391 -

TypeError: ignored

In [ ]:
print( test_model(model, "/content/jpg_k"+str(i)))

Found 3500 files belonging to 2 classes.
55/55 [==============================] - 15s 260ms/step - loss: 0.5576 - auc: 0.7755 - accuracy: 0.6586
[0.5575764775276184, 0.7754876613616943, 0.6585714221000671]


####k1

In [ ]:
i = 1
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/20
329/329 [==============================] - 309s 873ms/step - loss: 0.7254 - auc: 0.6357 - accuracy: 0.5860 - val_loss: 0.7913 - val_auc: 0.4740 - val_accuracy: 0.2306
Epoch 2/20
329/329 [==============================] - 289s 878ms/step - loss: 0.5946 - auc: 0.7366 - accuracy: 0.6296 - val_loss: 1.0491 - val_auc: 0.7055 - val_accuracy: 0.3812
Epoch 3/20
329/329 [==============================] - 290s 880ms/step - loss: 0.5563 - auc: 0.7778 - accuracy: 0.6498 - val_loss: 0.4300 - val_auc: 0.7639 - val_accuracy: 0.8021
Epoch 4/20
329/329 [==============================] - 290s 880ms/step - loss: 0.5329 - auc: 0.8003 - accuracy: 0.6700 - val_loss: 0.4020 - val_auc: 0.7781 - val_accuracy: 0.8225
Epoch 5/20
329/329 [==============================] - 291s 882ms/step - loss: 0.5130 - auc: 0.8191 - accuracy: 0.6950 - val_loss: 0.3515 -

In [ ]:
clear_train_dir()

####k2

In [ ]:
i = 2
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/20
329/329 [==============================] - 291s 866ms/step - loss: 0.7397 - auc: 0.6250 - accuracy: 0.5820 - val_loss: 0.6762 - val_auc: 0.5616 - val_accuracy: 0.5533
Epoch 2/20
329/329 [==============================] - 289s 876ms/step - loss: 0.6112 - auc: 0.7196 - accuracy: 0.6245 - val_loss: 0.9357 - val_auc: 0.6311 - val_accuracy: 0.4489
Epoch 3/20
329/329 [==============================] - 290s 880ms/step - loss: 0.5664 - auc: 0.7680 - accuracy: 0.6454 - val_loss: 0.4401 - val_auc: 0.7795 - val_accuracy: 0.7848
Epoch 4/20
329/329 [==============================] - 291s 882ms/step - loss: 0.5432 - auc: 0.7909 - accuracy: 0.6622 - val_loss: 0.4391 - val_auc: 0.7878 - val_accuracy: 0.7780
Epoch 5/20
329/329 [==============================] - 289s 878ms/step - loss: 0.5174 - auc: 0.8163 - accuracy: 0.6970 - val_loss: 0.4325 -

In [ ]:
clear_train_dir()

####k3

In [ ]:
i = 3
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/20
329/329 [==============================] - 295s 880ms/step - loss: 0.7332 - auc: 0.6313 - accuracy: 0.5837 - val_loss: 0.9474 - val_auc: 0.6264 - val_accuracy: 0.1400
Epoch 2/20
329/329 [==============================] - 289s 877ms/step - loss: 0.6025 - auc: 0.7289 - accuracy: 0.6235 - val_loss: 1.3519 - val_auc: 0.6086 - val_accuracy: 0.2096
Epoch 3/20
329/329 [==============================] - 289s 877ms/step - loss: 0.5649 - auc: 0.7643 - accuracy: 0.6315 - val_loss: 0.8170 - val_auc: 0.7795 - val_accuracy: 0.5323
Epoch 4/20
329/329 [==============================] - 289s 877ms/step - loss: 0.5363 - auc: 0.7946 - accuracy: 0.6597 - val_loss: 0.5427 - val_auc: 0.7929 - val_accuracy: 0.6499
Epoch 5/20
329/329 [==============================] - 289s 878ms/step - loss: 0.5163 - auc: 0.8142 - accuracy: 0.6839 - val_loss: 0.3704 -

In [ ]:
clear_train_dir()

####k4

In [ ]:
i = 4
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/20
329/329 [==============================] - 296s 882ms/step - loss: 0.7559 - auc: 0.6210 - accuracy: 0.5820 - val_loss: 0.7418 - val_auc: 0.5024 - val_accuracy: 0.2250
Epoch 2/20
329/329 [==============================] - 289s 877ms/step - loss: 0.6022 - auc: 0.7289 - accuracy: 0.6286 - val_loss: 1.3162 - val_auc: 0.6285 - val_accuracy: 0.2541
Epoch 3/20
329/329 [==============================] - 289s 878ms/step - loss: 0.5552 - auc: 0.7767 - accuracy: 0.6501 - val_loss: 0.8727 - val_auc: 0.7774 - val_accuracy: 0.5198
Epoch 4/20
329/329 [==============================] - 289s 878ms/step - loss: 0.5293 - auc: 0.8037 - accuracy: 0.6778 - val_loss: 1.0008 - val_auc: 0.7827 - val_accuracy: 0.4938
Epoch 5/20
329/329 [==============================] - 290s 880ms/step - loss: 0.5132 - auc: 0.8205 - accuracy: 0.6985 - val_loss: 0.3990 -

In [ ]:
clear_train_dir()

####k5

In [ ]:
i = 5
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/20
329/329 [==============================] - 296s 883ms/step - loss: 0.7154 - auc: 0.6361 - accuracy: 0.5886 - val_loss: 1.0780 - val_auc: 0.5507 - val_accuracy: 0.1400
Epoch 2/20
329/329 [==============================] - 290s 879ms/step - loss: 0.6075 - auc: 0.7229 - accuracy: 0.6217 - val_loss: 1.3979 - val_auc: 0.5796 - val_accuracy: 0.2497
Epoch 3/20
329/329 [==============================] - 290s 880ms/step - loss: 0.5579 - auc: 0.7748 - accuracy: 0.6435 - val_loss: 0.5957 - val_auc: 0.7790 - val_accuracy: 0.6125
Epoch 4/20
329/329 [==============================] - 289s 878ms/step - loss: 0.5440 - auc: 0.7900 - accuracy: 0.6631 - val_loss: 0.5285 - val_auc: 0.7929 - val_accuracy: 0.6661
Epoch 5/20
329/329 [==============================] - 290s 879ms/step - loss: 0.5159 - auc: 0.8161 - accuracy: 0.6911 - val_loss: 0.4611 -

In [ ]:
clear_train_dir()

####k6

In [ ]:
i = 6
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/20
329/329 [==============================] - 308s 870ms/step - loss: 0.7002 - auc: 0.6475 - accuracy: 0.5917 - val_loss: 0.5996 - val_auc: 0.5817 - val_accuracy: 0.8001
Epoch 2/20
329/329 [==============================] - 334s 1s/step - loss: 0.5961 - auc: 0.7347 - accuracy: 0.6292 - val_loss: 1.5340 - val_auc: 0.6227 - val_accuracy: 0.1971
Epoch 3/20
329/329 [==============================] - 291s 882ms/step - loss: 0.5526 - auc: 0.7786 - accuracy: 0.6485 - val_loss: 0.5883 - val_auc: 0.7704 - val_accuracy: 0.6375
Epoch 4/20
329/329 [==============================] - 293s 890ms/step - loss: 0.5367 - auc: 0.7975 - accuracy: 0.6699 - val_loss: 0.3996 - val_auc: 0.7837 - val_accuracy: 0.8136
Epoch 5/20
329/329 [==============================] - 293s 889ms/step - loss: 0.5050 - auc: 0.8233 - accuracy: 0.6970 - val_loss: 0.6129 - va

In [ ]:
clear_train_dir()

####k7

In [ ]:
i = 7
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/20
329/329 [==============================] - 322s 922ms/step - loss: 0.7322 - auc: 0.6255 - accuracy: 0.5837 - val_loss: 0.6405 - val_auc: 0.6100 - val_accuracy: 0.7059
Epoch 2/20
329/329 [==============================] - 305s 926ms/step - loss: 0.6086 - auc: 0.7241 - accuracy: 0.6238 - val_loss: 1.8447 - val_auc: 0.5268 - val_accuracy: 0.1693
Epoch 3/20
329/329 [==============================] - 305s 927ms/step - loss: 0.5710 - auc: 0.7609 - accuracy: 0.6374 - val_loss: 0.6804 - val_auc: 0.7823 - val_accuracy: 0.5601
Epoch 4/20
329/329 [==============================] - 305s 927ms/step - loss: 0.5464 - auc: 0.7893 - accuracy: 0.6633 - val_loss: 0.4238 - val_auc: 0.8026 - val_accuracy: 0.7951
Epoch 5/20
329/329 [==============================] - 305s 926ms/step - loss: 0.5224 - auc: 0.8111 - accuracy: 0.6898 - val_loss: 0.5576 -

In [ ]:
clear_train_dir()

####k8

In [ ]:
i = 8
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/20
329/329 [==============================] - 312s 930ms/step - loss: 0.7166 - auc: 0.6342 - accuracy: 0.5857 - val_loss: 0.4516 - val_auc: 0.4621 - val_accuracy: 0.8600
Epoch 2/20
329/329 [==============================] - 306s 927ms/step - loss: 0.5983 - auc: 0.7307 - accuracy: 0.6249 - val_loss: 2.0993 - val_auc: 0.5181 - val_accuracy: 0.1774
Epoch 3/20
329/329 [==============================] - 306s 927ms/step - loss: 0.5556 - auc: 0.7760 - accuracy: 0.6483 - val_loss: 0.4357 - val_auc: 0.7438 - val_accuracy: 0.8096
Epoch 4/20
329/329 [==============================] - 306s 928ms/step - loss: 0.5304 - auc: 0.8033 - accuracy: 0.6739 - val_loss: 0.4345 - val_auc: 0.7737 - val_accuracy: 0.7917
Epoch 5/20
329/329 [==============================] - 306s 928ms/step - loss: 0.5111 - auc: 0.8211 - accuracy: 0.7029 - val_loss: 0.4211 -

In [ ]:
clear_train_dir()

####k9

In [ ]:
i = 9
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31500 files belonging to 2 classes.
Using 21000 files for training.
Found 31500 files belonging to 2 classes.
Using 10500 files for validation.
Epoch 1/20
329/329 [==============================] - 313s 933ms/step - loss: 0.7182 - auc: 0.6361 - accuracy: 0.5910 - val_loss: 0.4035 - val_auc: 0.5633 - val_accuracy: 0.8588
Epoch 2/20
329/329 [==============================] - 305s 927ms/step - loss: 0.5994 - auc: 0.7317 - accuracy: 0.6338 - val_loss: 1.5161 - val_auc: 0.6427 - val_accuracy: 0.1999
Epoch 3/20
329/329 [==============================] - 306s 929ms/step - loss: 0.5590 - auc: 0.7711 - accuracy: 0.6462 - val_loss: 0.4225 - val_auc: 0.7753 - val_accuracy: 0.8181
Epoch 4/20
329/329 [==============================] - 306s 927ms/step - loss: 0.5328 - auc: 0.7982 - accuracy: 0.6673 - val_loss: 0.3544 - val_auc: 0.7829 - val_accuracy: 0.8550
Epoch 5/20
329/329 [==============================] - 305s 927ms/step - loss: 0.5022 - auc: 0.8254 - accuracy: 0.7009 - val_loss: 0.3560 -

In [ ]:
clear_train_dir()

### [ok] Baseline W/ Transfered Learning

#### Declarações

In [ ]:
def clear_train_dir():
  for file in os.listdir("/content/train/yes/"):
    os.remove("/content/train/yes/" + file)

  for file in os.listdir("/content/train/no/"):
    os.remove("/content/train/no/" + file)

In [ ]:
def load_train_dir(ki):
  k = [0,1,2,3,4,5,6,7,8,9]
  for i in range(10):
    if i!= k[ki]:
      yesdir = "/content/jpg_k"+str(i)+"/yes/"
      nodir = "/content/jpg_k"+str(i)+"/no/"
      for file in os.listdir(yesdir):
        shutil.copy(yesdir + file, "/content/train/yes")
      for file in os.listdir(nodir):
        shutil.copy(nodir + file, "/content/train/no")

In [ ]:
def train_model(w0, w1):
    train_data = image_dataset_from_directory(
      ("/content/train"),
      labels='inferred',
      validation_split=1/3,
      subset="training",
      seed=123,
      batch_size = 64,
      color_mode = "rgb"
    )
  
    val_data = image_dataset_from_directory(
      "/content/train",
      labels='inferred',
      validation_split=1/3,
      subset="validation",
      seed=123,
      batch_size = 64,
      color_mode = "rgb"
    )

    class_names = train_data.class_names

    normalization_layer = Rescaling(1./255)

    normalized_data = train_data.map(lambda x, y: (normalization_layer(x), y))
    image_batch, labels_batch = next(iter(normalized_data))

    normalized_val_data = val_data.map(lambda x, y: (normalization_layer(x), y))
    val_image_batch, val_labels_batch = next(iter(normalized_val_data))

    keras.backend.clear_session()

    resnet_model = Sequential()

    resnet = tf.keras.applications.resnet50.ResNet50(
        include_top=False,
        input_shape=image_batch.shape[1:],
        pooling='avg',
        weights='imagenet',
        classes=2,
    )

    resnet_model.add(resnet)

    resnet_model.add(Flatten())

    resnet_model.add(Dense(512, activation="relu"))

    resnet_model.add(Dropout(0.5))

    resnet_model.add(Dense(1, activation="sigmoid"))

    resnet_model.compile(loss='binary_crossentropy', 
                         optimizer=optimizers.Adam(learning_rate=0.00001),
                         metrics=[keras.metrics.AUC(), 'accuracy'])
  
    run_hist = resnet_model.fit(
      normalized_data, 
      validation_data=normalized_val_data, epochs=20, class_weight={0:w0, 1:w1}, 
      callbacks=[keras.callbacks.EarlyStopping(
                      monitor="val_auc",
                      min_delta=0,
                      patience=5,
                      verbose=1,
                      mode="auto",
                      baseline=None,
                      restore_best_weights=True,
                  )]
    )

    return resnet_model

In [ ]:
def test_model(resnet_model, test_path):
    test_data = image_dataset_from_directory(
      test_path,
      labels='inferred',
      label_mode="int",
      batch_size = 64,
      color_mode = "rgb"
    )

    normalization_layer = Rescaling(1./255)

    normalized_test_data = test_data.map(lambda x, y: (normalization_layer(x), y))
    test_image_batch, test_labels_batch = next(iter(normalized_test_data))

    results = resnet_model.evaluate(normalized_test_data)
    return results

In [ ]:
neg = 27062
pos = 4437
total = pos+neg

w0 = (1 / neg) * (total / 2.0)
w1 = (1 / pos) * (total / 2.0)

print('Peso para a classe 0: {:.2f}'.format(w0))
print('Peso para a classe 1: {:.2f}'.format(w1))

Peso para a classe 0: 0.58
Peso para a classe 1: 3.55


In [ ]:
print('Numero de imagens: NO')
!ls /content/train/no | wc -l
print('Numero de imagens: YES')
!ls /content/train/yes | wc -l

Numero de imagens: NO
0
Numero de imagens: YES
0


#### k0

In [ ]:
i = 0
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
94765736/94765736 [==============================] - 1s 0us/step
Epoch 1/20
329/329 [==============================] - 315s 901ms/step - loss: 0.5501 - auc: 0.7880 - accuracy: 0.6703 - val_loss: 0.4162 - val_auc: 0.6390 - val_accuracy: 0.8565
Epoch 2/20
329/329 [==============================] - 299s 909ms/step - loss: 0.4284 - auc: 0.8808 - accuracy: 0.7769 - val_loss: 1.1079 - val_auc: 0.7079 - val_accuracy: 0.4393
Epoch 3/20
329/329 [==============================] - 299s 908ms/step - loss: 0.3225 - auc: 0.9362 - accuracy: 0.8479 - val_loss: 1.1662 - val_auc: 0.7762 - val_accuracy: 0.5503
Epoch 4/20
329/329 [==============================] - 299s 909ms/step - loss: 0.2098 - auc: 0.9741 - accuracy: 0.9072 - val_loss: 1.0406 - val_auc: 0.7920 - val_accuracy: 0.6263
Epoch 5/20
329/329 [==============================] - 340s 1s/step - loss

In [ ]:
clear_train_dir()

####k1

In [ ]:
i = 1
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/20
329/329 [==============================] - 306s 912ms/step - loss: 0.5623 - auc: 0.7763 - accuracy: 0.6601 - val_loss: 0.3977 - val_auc: 0.6420 - val_accuracy: 0.8601
Epoch 2/20
329/329 [==============================] - 300s 910ms/step - loss: 0.4421 - auc: 0.8706 - accuracy: 0.7570 - val_loss: 0.5824 - val_auc: 0.6819 - val_accuracy: 0.7018
Epoch 3/20
329/329 [==============================] - 301s 912ms/step - loss: 0.3430 - auc: 0.9265 - accuracy: 0.8334 - val_loss: 1.4127 - val_auc: 0.7647 - val_accuracy: 0.5044
Epoch 4/20
329/329 [==============================] - 341s 1s/step - loss: 0.2380 - auc: 0.9665 - accuracy: 0.8930 - val_loss: 1.2425 - val_auc: 0.7968 - val_accuracy: 0.6129
Epoch 5/20
329/329 [==============================] - 300s 911ms/step - loss: 0.1325 - auc: 0.9903 - accuracy: 0.9450 - val_loss: 1.3686 - va

In [ ]:
clear_train_dir()

####k2

In [ ]:
i = 2
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/20
329/329 [==============================] - 306s 912ms/step - loss: 0.5637 - auc: 0.7758 - accuracy: 0.6631 - val_loss: 0.4115 - val_auc: 0.5954 - val_accuracy: 0.8600
Epoch 2/20
329/329 [==============================] - 299s 908ms/step - loss: 0.4423 - auc: 0.8723 - accuracy: 0.7621 - val_loss: 0.7089 - val_auc: 0.7019 - val_accuracy: 0.5785
Epoch 3/20
329/329 [==============================] - 340s 1s/step - loss: 0.3381 - auc: 0.9294 - accuracy: 0.8405 - val_loss: 1.0870 - val_auc: 0.7808 - val_accuracy: 0.6000
Epoch 4/20
329/329 [==============================] - 301s 912ms/step - loss: 0.2282 - auc: 0.9692 - accuracy: 0.9026 - val_loss: 1.3372 - val_auc: 0.8190 - val_accuracy: 0.6623
Epoch 5/20
329/329 [==============================] - 340s 1s/step - loss: 0.1265 - auc: 0.9911 - accuracy: 0.9509 - val_loss: 0.5023 - val_a

In [ ]:
clear_train_dir()

####k3

In [ ]:
i = 3
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
94765736/94765736 [==============================] - 3s 0us/step
Epoch 1/20
329/329 [==============================] - 315s 901ms/step - loss: 0.5489 - auc: 0.7912 - accuracy: 0.6785 - val_loss: 0.4603 - val_auc: 0.4319 - val_accuracy: 0.8598
Epoch 2/20
329/329 [==============================] - 300s 912ms/step - loss: 0.4285 - auc: 0.8806 - accuracy: 0.7800 - val_loss: 0.7935 - val_auc: 0.6922 - val_accuracy: 0.5542
Epoch 3/20
329/329 [==============================] - 300s 912ms/step - loss: 0.3187 - auc: 0.9380 - accuracy: 0.8528 - val_loss: 1.1971 - val_auc: 0.7773 - val_accuracy: 0.5959
Epoch 4/20
329/329 [==============================] - 300s 911ms/step - loss: 0.2110 - auc: 0.9742 - accuracy: 0.9141 - val_loss: 1.6234 - val_auc: 0.7925 - val_accuracy: 0.6628
Epoch 5/20
329/329 [==============================] - 301s 913ms/step - l

In [ ]:
clear_train_dir()

####k4

In [ ]:
i = 4
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/20
329/329 [==============================] - 347s 1s/step - loss: 0.5489 - auc: 0.7869 - accuracy: 0.6689 - val_loss: 0.4379 - val_auc: 0.6301 - val_accuracy: 0.8377
Epoch 2/20
329/329 [==============================] - 301s 912ms/step - loss: 0.4287 - auc: 0.8797 - accuracy: 0.7691 - val_loss: 1.6145 - val_auc: 0.6974 - val_accuracy: 0.2495
Epoch 3/20
329/329 [==============================] - 301s 914ms/step - loss: 0.3292 - auc: 0.9330 - accuracy: 0.8393 - val_loss: 2.7176 - val_auc: 0.7460 - val_accuracy: 0.4840
Epoch 4/20
329/329 [==============================] - 301s 913ms/step - loss: 0.2148 - auc: 0.9733 - accuracy: 0.9047 - val_loss: 3.6023 - val_auc: 0.7539 - val_accuracy: 0.5315
Epoch 5/20
329/329 [==============================] - 300s 912ms/step - loss: 0.1189 - auc: 0.9921 - accuracy: 0.9525 - val_loss: 3.0734 - va

In [ ]:
clear_train_dir()

####k5

In [ ]:
i = 5
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/20
329/329 [==============================] - 307s 914ms/step - loss: 0.5629 - auc: 0.7771 - accuracy: 0.6634 - val_loss: 0.4380 - val_auc: 0.6423 - val_accuracy: 0.8600
Epoch 2/20
329/329 [==============================] - 300s 911ms/step - loss: 0.4419 - auc: 0.8713 - accuracy: 0.7629 - val_loss: 0.4323 - val_auc: 0.7016 - val_accuracy: 0.7986
Epoch 3/20
329/329 [==============================] - 341s 1s/step - loss: 0.3396 - auc: 0.9286 - accuracy: 0.8408 - val_loss: 1.5619 - val_auc: 0.7657 - val_accuracy: 0.5345
Epoch 4/20
329/329 [==============================] - 301s 912ms/step - loss: 0.2370 - auc: 0.9665 - accuracy: 0.8940 - val_loss: 2.1457 - val_auc: 0.7892 - val_accuracy: 0.5650
Epoch 5/20
329/329 [==============================] - 341s 1s/step - loss: 0.1273 - auc: 0.9907 - accuracy: 0.9479 - val_loss: 1.7440 - val_a

In [ ]:
clear_train_dir()

####k6

In [ ]:
i = 6
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
94765736/94765736 [==============================] - 1s 0us/step
Epoch 1/20
329/329 [==============================] - 306s 877ms/step - loss: 0.5544 - auc: 0.7829 - accuracy: 0.6664 - val_loss: 0.4632 - val_auc: 0.6755 - val_accuracy: 0.7684
Epoch 2/20
329/329 [==============================] - 290s 880ms/step - loss: 0.4285 - auc: 0.8811 - accuracy: 0.7800 - val_loss: 1.2183 - val_auc: 0.7274 - val_accuracy: 0.3160
Epoch 3/20
329/329 [==============================] - 290s 880ms/step - loss: 0.3207 - auc: 0.9372 - accuracy: 0.8519 - val_loss: 1.9023 - val_auc: 0.7549 - val_accuracy: 0.5044
Epoch 4/20
329/329 [==============================] - 292s 883ms/step - loss: 0.2102 - auc: 0.9742 - accuracy: 0.9135 - val_loss: 2.3591 - val_auc: 0.7764 - val_accuracy: 0.5682
Epoch 5/20
329/329 [==============================] - 290s 879ms/step - l

In [ ]:
clear_train_dir()

####k7

In [ ]:
i = 7
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/20
329/329 [==============================] - 298s 888ms/step - loss: 0.5554 - auc: 0.7811 - accuracy: 0.6638 - val_loss: 0.4114 - val_auc: 0.6031 - val_accuracy: 0.8600
Epoch 2/20
329/329 [==============================] - 292s 884ms/step - loss: 0.4333 - auc: 0.8769 - accuracy: 0.7690 - val_loss: 0.9163 - val_auc: 0.7117 - val_accuracy: 0.4806
Epoch 3/20
329/329 [==============================] - 292s 886ms/step - loss: 0.3278 - auc: 0.9337 - accuracy: 0.8453 - val_loss: 1.4698 - val_auc: 0.7788 - val_accuracy: 0.4995
Epoch 4/20
329/329 [==============================] - 292s 885ms/step - loss: 0.2252 - auc: 0.9696 - accuracy: 0.8985 - val_loss: 1.0804 - val_auc: 0.8122 - val_accuracy: 0.6165
Epoch 5/20
329/329 [==============================] - 291s 881ms/step - loss: 0.1254 - auc: 0.9910 - accuracy: 0.9450 - val_loss: 0.5524 -

In [ ]:
clear_train_dir()

####k8

In [ ]:
i = 8
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/20
329/329 [==============================] - 297s 884ms/step - loss: 0.5641 - auc: 0.7741 - accuracy: 0.6629 - val_loss: 0.5037 - val_auc: 0.4669 - val_accuracy: 0.8580
Epoch 2/20
329/329 [==============================] - 290s 880ms/step - loss: 0.4441 - auc: 0.8692 - accuracy: 0.7537 - val_loss: 0.5205 - val_auc: 0.6349 - val_accuracy: 0.7841
Epoch 3/20
329/329 [==============================] - 291s 882ms/step - loss: 0.3423 - auc: 0.9270 - accuracy: 0.8320 - val_loss: 1.5044 - val_auc: 0.7706 - val_accuracy: 0.5451
Epoch 4/20
329/329 [==============================] - 290s 881ms/step - loss: 0.2358 - auc: 0.9673 - accuracy: 0.8928 - val_loss: 1.8870 - val_auc: 0.7826 - val_accuracy: 0.6044
Epoch 5/20
329/329 [==============================] - 291s 881ms/step - loss: 0.1209 - auc: 0.9923 - accuracy: 0.9501 - val_loss: 1.8205 -

In [ ]:
clear_train_dir()

####k9

In [ ]:
i = 9
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31500 files belonging to 2 classes.
Using 21000 files for training.
Found 31500 files belonging to 2 classes.
Using 10500 files for validation.
94765736/94765736 [==============================] - 1s 0us/step
Epoch 1/20
329/329 [==============================] - 312s 893ms/step - loss: 0.5580 - auc: 0.7825 - accuracy: 0.6669 - val_loss: 0.5076 - val_auc: 0.5972 - val_accuracy: 0.7819
Epoch 2/20
329/329 [==============================] - 296s 898ms/step - loss: 0.4219 - auc: 0.8840 - accuracy: 0.7764 - val_loss: 1.2744 - val_auc: 0.6762 - val_accuracy: 0.3386
Epoch 3/20
329/329 [==============================] - 295s 895ms/step - loss: 0.3226 - auc: 0.9362 - accuracy: 0.8501 - val_loss: 2.3061 - val_auc: 0.7530 - val_accuracy: 0.5036
Epoch 4/20
329/329 [==============================] - 295s 896ms/step - loss: 0.2140 - auc: 0.9733 - accuracy: 0.9068 - val_loss: 2.0282 - val_auc: 0.7790 - val_accuracy: 0.5704
Epoch 5/20
329/329 [==============================] - 296s 897ms/step - l

In [ ]:
clear_train_dir()

### [ok] Baseline: W/ Tranfered Learning & W/ Data Aug

In [ ]:
from keras.layers import RandomFlip, RandomRotation, RandomCrop

#### Declarações

In [ ]:
def clear_train_dir():
  for file in os.listdir("/content/train/yes/"):
    os.remove("/content/train/yes/" + file)

  for file in os.listdir("/content/train/no/"):
    os.remove("/content/train/no/" + file)

In [ ]:
def load_train_dir(ki):
  k = [0,1,2,3,4,5,6,7,8,9]
  for i in range(10):
    if i!= k[ki]:
      yesdir = "/content/jpg_k"+str(i)+"/yes/"
      nodir = "/content/jpg_k"+str(i)+"/no/"
      for file in os.listdir(yesdir):
        shutil.copy(yesdir + file, "/content/train/yes")
      for file in os.listdir(nodir):
        shutil.copy(nodir + file, "/content/train/no")

In [ ]:
def train_model(w0, w1):
    train_data = image_dataset_from_directory(
      ("/content/train"),
      labels='inferred',
      validation_split=1/3,
      subset="training",
      seed=123,
      batch_size = 64,
      color_mode = "rgb"
    )
  
    val_data = image_dataset_from_directory(
      "/content/train",
      labels='inferred',
      validation_split=1/3,
      subset="validation",
      seed=123,
      batch_size = 64,
      color_mode = "rgb"
    )

    class_names = train_data.class_names

    normalization_layer = Rescaling(1./255)

    normalized_data = train_data.map(lambda x, y: (normalization_layer(x), y))
    image_batch, labels_batch = next(iter(normalized_data))

    normalized_val_data = val_data.map(lambda x, y: (normalization_layer(x), y))
    val_image_batch, val_labels_batch = next(iter(normalized_val_data))

    keras.backend.clear_session()

    resnet_model = Sequential()

    resnet_model.add(RandomFlip("horizontal"))
    resnet_model.add(RandomRotation(0.2))
    resnet_model.add(RandomCrop(256, 256, seed=1)) 

    resnet = tf.keras.applications.resnet50.ResNet50(
        include_top=False,
        input_shape=image_batch.shape[1:],
        pooling='avg',
        weights='imagenet',
        classes=2,
    )

    resnet_model.add(resnet)

    resnet_model.add(Flatten())

    resnet_model.add(Dense(512, activation="relu"))

    resnet_model.add(Dropout(0.5))

    resnet_model.add(Dense(1, activation="sigmoid"))

    resnet_model.compile(loss='binary_crossentropy', 
                         optimizer=optimizers.Adam(learning_rate=0.00001),
                         metrics=[keras.metrics.AUC(), 'accuracy'])
  
    run_hist = resnet_model.fit(
      normalized_data, 
      validation_data=normalized_val_data, epochs=100, class_weight={0:w0, 1:w1}, 
      callbacks=[keras.callbacks.EarlyStopping(
                      monitor="val_auc",
                      min_delta=0,
                      patience=5,
                      verbose=1,
                      mode="auto",
                      baseline=None,
                      restore_best_weights=True,
                  )]
    )

    return resnet_model

In [ ]:
def test_model(resnet_model, test_path):
    test_data = image_dataset_from_directory(
      test_path,
      labels='inferred',
      label_mode="int",
      batch_size = 64,
      color_mode = "rgb"
    )

    normalization_layer = Rescaling(1./255)

    normalized_test_data = test_data.map(lambda x, y: (normalization_layer(x), y))
    test_image_batch, test_labels_batch = next(iter(normalized_test_data))

    results = resnet_model.evaluate(normalized_test_data)
    return results

In [ ]:
neg = 27062
pos = 4437
total = pos+neg

w0 = (1 / neg) * (total / 2.0)
w1 = (1 / pos) * (total / 2.0)

print('Peso para a classe 0: {:.2f}'.format(w0))
print('Peso para a classe 1: {:.2f}'.format(w1))

Peso para a classe 0: 0.58
Peso para a classe 1: 3.55


In [ ]:
print('Numero de imagens: NO')
!ls /content/train/no | wc -l
print('Numero de imagens: YES')
!ls /content/train/yes | wc -l

Numero de imagens: NO
0
Numero de imagens: YES
0


#### k0

In [ ]:
i = 0
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/100
329/329 [==============================] - 325s 966ms/step - loss: 0.5732 - auc: 0.7668 - accuracy: 0.6530 - val_loss: 0.4280 - val_auc: 0.6378 - val_accuracy: 0.8576
Epoch 2/100
329/329 [==============================] - 316s 959ms/step - loss: 0.4968 - auc: 0.8342 - accuracy: 0.7198 - val_loss: 1.1859 - val_auc: 0.7020 - val_accuracy: 0.3536
Epoch 3/100
329/329 [==============================] - 317s 961ms/step - loss: 0.4552 - auc: 0.8650 - accuracy: 0.7700 - val_loss: 1.7347 - val_auc: 0.8004 - val_accuracy: 0.4336
Epoch 4/100
329/329 [==============================] - 317s 962ms/step - loss: 0.4246 - auc: 0.8843 - accuracy: 0.7963 - val_loss: 1.2404 - val_auc: 0.8228 - val_accuracy: 0.5762
Epoch 5/100
329/329 [==============================] - 319s 966ms/step - loss: 0.3990 - auc: 0.8998 - accuracy: 0.8179 - val_loss: 1.1

In [ ]:
clear_train_dir()

####k1

In [ ]:
i = 1
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
94765736/94765736 [==============================] - 1s 0us/step
Epoch 1/100
329/329 [==============================] - 336s 963ms/step - loss: 0.5647 - auc: 0.7702 - accuracy: 0.6564 - val_loss: 0.7629 - val_auc: 0.6689 - val_accuracy: 0.4645
Epoch 2/100
329/329 [==============================] - 320s 972ms/step - loss: 0.4971 - auc: 0.8333 - accuracy: 0.7144 - val_loss: 1.5925 - val_auc: 0.6729 - val_accuracy: 0.2182
Epoch 3/100
329/329 [==============================] - 321s 973ms/step - loss: 0.4548 - auc: 0.8647 - accuracy: 0.7642 - val_loss: 1.7552 - val_auc: 0.7717 - val_accuracy: 0.3974
Epoch 4/100
329/329 [==============================] - 363s 1s/step - loss: 0.4296 - auc: 0.8822 - accuracy: 0.7918 - val_loss: 1.2611 - val_auc: 0.8095 - val_accuracy: 0.5554
Epoch 5/100
329/329 [==============================] - 321s 973ms/step -

In [ ]:
clear_train_dir()

####k2

In [ ]:
i = 2
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/100
329/329 [==============================] - 330s 978ms/step - loss: 0.5719 - auc: 0.7661 - accuracy: 0.6534 - val_loss: 0.4535 - val_auc: 0.6471 - val_accuracy: 0.8600
Epoch 2/100
329/329 [==============================] - 321s 973ms/step - loss: 0.4947 - auc: 0.8373 - accuracy: 0.7272 - val_loss: 0.5477 - val_auc: 0.7079 - val_accuracy: 0.6905
Epoch 3/100
329/329 [==============================] - 321s 973ms/step - loss: 0.4551 - auc: 0.8659 - accuracy: 0.7760 - val_loss: 1.6455 - val_auc: 0.7735 - val_accuracy: 0.5154
Epoch 4/100
329/329 [==============================] - 322s 977ms/step - loss: 0.4300 - auc: 0.8821 - accuracy: 0.8020 - val_loss: 1.7805 - val_auc: 0.8033 - val_accuracy: 0.5419
Epoch 5/100
329/329 [==============================] - 321s 973ms/step - loss: 0.4071 - auc: 0.8958 - accuracy: 0.8157 - val_loss: 1.0

In [ ]:
clear_train_dir()

####k3

In [ ]:
i = 3
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
94765736/94765736 [==============================] - 0s 0us/step
Epoch 1/100
329/329 [==============================] - 336s 961ms/step - loss: 0.5688 - auc: 0.7674 - accuracy: 0.6487 - val_loss: 0.4055 - val_auc: 0.5564 - val_accuracy: 0.8600
Epoch 2/100
329/329 [==============================] - 321s 973ms/step - loss: 0.4947 - auc: 0.8364 - accuracy: 0.7215 - val_loss: 0.7068 - val_auc: 0.6683 - val_accuracy: 0.5434
Epoch 3/100
329/329 [==============================] - 320s 972ms/step - loss: 0.4564 - auc: 0.8641 - accuracy: 0.7618 - val_loss: 1.7069 - val_auc: 0.7895 - val_accuracy: 0.4391
Epoch 4/100
329/329 [==============================] - 320s 971ms/step - loss: 0.4246 - auc: 0.8850 - accuracy: 0.7943 - val_loss: 1.5298 - val_auc: 0.8043 - val_accuracy: 0.5677
Epoch 5/100
329/329 [==============================] - 320s 970ms/ste

In [ ]:
clear_train_dir()

####k4

In [ ]:
i = 4
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/100
329/329 [==============================] - 330s 979ms/step - loss: 0.5716 - auc: 0.7643 - accuracy: 0.6505 - val_loss: 0.4900 - val_auc: 0.6271 - val_accuracy: 0.8600
Epoch 2/100
329/329 [==============================] - 321s 974ms/step - loss: 0.5037 - auc: 0.8283 - accuracy: 0.7066 - val_loss: 0.3816 - val_auc: 0.7175 - val_accuracy: 0.8523
Epoch 3/100
329/329 [==============================] - 361s 1s/step - loss: 0.4722 - auc: 0.8539 - accuracy: 0.7476 - val_loss: 1.1506 - val_auc: 0.7735 - val_accuracy: 0.5989
Epoch 4/100
329/329 [==============================] - 362s 1s/step - loss: 0.4364 - auc: 0.8774 - accuracy: 0.7843 - val_loss: 1.1406 - val_auc: 0.8059 - val_accuracy: 0.6324
Epoch 5/100
329/329 [==============================] - 322s 977ms/step - loss: 0.4150 - auc: 0.8902 - accuracy: 0.7978 - val_loss: 1.0269 - 

In [ ]:
clear_train_dir()

####k5

In [ ]:
i = 5
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
94765736/94765736 [==============================] - 5s 0us/step
Epoch 1/100
329/329 [==============================] - 340s 971ms/step - loss: 0.5819 - auc: 0.7563 - accuracy: 0.6454 - val_loss: 0.4322 - val_auc: 0.5973 - val_accuracy: 0.8600
Epoch 2/100
329/329 [==============================] - 322s 976ms/step - loss: 0.5036 - auc: 0.8284 - accuracy: 0.7140 - val_loss: 0.6511 - val_auc: 0.7056 - val_accuracy: 0.6259
Epoch 3/100
329/329 [==============================] - 322s 977ms/step - loss: 0.4560 - auc: 0.8660 - accuracy: 0.7696 - val_loss: 1.0296 - val_auc: 0.7943 - val_accuracy: 0.5302
Epoch 4/100
329/329 [==============================] - 322s 976ms/step - loss: 0.4317 - auc: 0.8810 - accuracy: 0.7969 - val_loss: 0.9951 - val_auc: 0.8325 - val_accuracy: 0.6097
Epoch 5/100
329/329 [==============================] - 321s 974ms/ste

In [ ]:
clear_train_dir()

####k6

In [ ]:
i = 6
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/100
329/329 [==============================] - 329s 979ms/step - loss: 0.5720 - auc: 0.7677 - accuracy: 0.6568 - val_loss: 0.3968 - val_auc: 0.6156 - val_accuracy: 0.8600
Epoch 2/100
329/329 [==============================] - 321s 974ms/step - loss: 0.4891 - auc: 0.8407 - accuracy: 0.7321 - val_loss: 0.8517 - val_auc: 0.6414 - val_accuracy: 0.4599
Epoch 3/100
329/329 [==============================] - 321s 975ms/step - loss: 0.4443 - auc: 0.8728 - accuracy: 0.7769 - val_loss: 1.7620 - val_auc: 0.7819 - val_accuracy: 0.4828
Epoch 4/100
329/329 [==============================] - 321s 975ms/step - loss: 0.4168 - auc: 0.8896 - accuracy: 0.8044 - val_loss: 1.2867 - val_auc: 0.8205 - val_accuracy: 0.5512
Epoch 5/100
329/329 [==============================] - 321s 974ms/step - loss: 0.3856 - auc: 0.9068 - accuracy: 0.8255 - val_loss: 1.0

In [ ]:
clear_train_dir()

####k7

In [ ]:
i = 7
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
94765736/94765736 [==============================] - 0s 0us/step
Epoch 1/100
329/329 [==============================] - 334s 954ms/step - loss: 0.5773 - auc: 0.7624 - accuracy: 0.6520 - val_loss: 0.4182 - val_auc: 0.6166 - val_accuracy: 0.8433
Epoch 2/100
329/329 [==============================] - 316s 959ms/step - loss: 0.5018 - auc: 0.8296 - accuracy: 0.7183 - val_loss: 0.8841 - val_auc: 0.7177 - val_accuracy: 0.4420
Epoch 3/100
329/329 [==============================] - 317s 961ms/step - loss: 0.4608 - auc: 0.8611 - accuracy: 0.7672 - val_loss: 1.8568 - val_auc: 0.7929 - val_accuracy: 0.4381
Epoch 4/100
329/329 [==============================] - 317s 962ms/step - loss: 0.4321 - auc: 0.8797 - accuracy: 0.7911 - val_loss: 1.3770 - val_auc: 0.8211 - val_accuracy: 0.5352
Epoch 5/100
329/329 [==============================] - 316s 960ms/ste

In [ ]:
clear_train_dir()

####k8

In [ ]:
i = 8
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/100
329/329 [==============================] - 324s 963ms/step - loss: 0.5836 - auc: 0.7543 - accuracy: 0.6422 - val_loss: 0.4957 - val_auc: 0.5490 - val_accuracy: 0.8571
Epoch 2/100
329/329 [==============================] - 316s 959ms/step - loss: 0.4999 - auc: 0.8311 - accuracy: 0.7198 - val_loss: 0.6624 - val_auc: 0.7113 - val_accuracy: 0.6213
Epoch 3/100
329/329 [==============================] - 316s 959ms/step - loss: 0.4569 - auc: 0.8645 - accuracy: 0.7730 - val_loss: 1.3555 - val_auc: 0.7830 - val_accuracy: 0.5559
Epoch 4/100
329/329 [==============================] - 316s 960ms/step - loss: 0.4250 - auc: 0.8852 - accuracy: 0.8033 - val_loss: 0.9828 - val_auc: 0.8242 - val_accuracy: 0.6277
Epoch 5/100
329/329 [==============================] - 317s 960ms/step - loss: 0.3986 - auc: 0.8995 - accuracy: 0.8184 - val_loss: 0.6

In [ ]:
clear_train_dir()

####k9

In [ ]:
i = 9
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31500 files belonging to 2 classes.
Using 21000 files for training.
Found 31500 files belonging to 2 classes.
Using 10500 files for validation.
Epoch 1/100
329/329 [==============================] - 324s 963ms/step - loss: 0.5706 - auc: 0.7659 - accuracy: 0.6578 - val_loss: 1.0757 - val_auc: 0.6253 - val_accuracy: 0.1414
Epoch 2/100
329/329 [==============================] - 316s 959ms/step - loss: 0.4967 - auc: 0.8334 - accuracy: 0.7144 - val_loss: 2.1823 - val_auc: 0.7311 - val_accuracy: 0.1831
Epoch 3/100
329/329 [==============================] - 316s 959ms/step - loss: 0.4574 - auc: 0.8642 - accuracy: 0.7630 - val_loss: 2.3115 - val_auc: 0.7612 - val_accuracy: 0.4194
Epoch 4/100
329/329 [==============================] - 316s 958ms/step - loss: 0.4300 - auc: 0.8807 - accuracy: 0.7908 - val_loss: 1.4358 - val_auc: 0.8080 - val_accuracy: 0.5821
Epoch 5/100
329/329 [==============================] - 316s 959ms/step - loss: 0.4065 - auc: 0.8953 - accuracy: 0.8088 - val_loss: 1.2

In [ ]:
clear_train_dir()

### [ok] Baseline: WO/ Tranfered Learning & W/ Data Aug

In [ ]:
from keras.layers import RandomFlip, RandomRotation, RandomCrop

#### Declarações

In [ ]:
def clear_train_dir():
  for file in os.listdir("/content/train/yes/"):
    os.remove("/content/train/yes/" + file)

  for file in os.listdir("/content/train/no/"):
    os.remove("/content/train/no/" + file)

In [ ]:
def load_train_dir(ki):
  k = [0,1,2,3,4,5,6,7,8,9]
  for i in range(10):
    if i!= k[ki]:
      yesdir = "/content/jpg_k"+str(i)+"/yes/"
      nodir = "/content/jpg_k"+str(i)+"/no/"
      for file in os.listdir(yesdir):
        shutil.copy(yesdir + file, "/content/train/yes")
      for file in os.listdir(nodir):
        shutil.copy(nodir + file, "/content/train/no")

In [ ]:
def train_model(w0, w1):
    train_data = image_dataset_from_directory(
      ("/content/train"),
      labels='inferred',
      validation_split=1/3,
      subset="training",
      seed=123,
      batch_size = 64,
      color_mode = "rgb"
    )
  
    val_data = image_dataset_from_directory(
      "/content/train",
      labels='inferred',
      validation_split=1/3,
      subset="validation",
      seed=123,
      batch_size = 64,
      color_mode = "rgb"
    )

    class_names = train_data.class_names

    normalization_layer = Rescaling(1./255)

    normalized_data = train_data.map(lambda x, y: (normalization_layer(x), y))
    image_batch, labels_batch = next(iter(normalized_data))

    normalized_val_data = val_data.map(lambda x, y: (normalization_layer(x), y))
    val_image_batch, val_labels_batch = next(iter(normalized_val_data))

    keras.backend.clear_session()

    resnet_model = Sequential()

    resnet_model.add(RandomFlip("horizontal"))
    resnet_model.add(RandomRotation(0.2))
    resnet_model.add(RandomCrop(256, 256, seed=1)) 

    resnet = tf.keras.applications.resnet50.ResNet50(
        include_top=False,
        input_shape=image_batch.shape[1:],
        pooling='avg',
        weights=None,
        classes=2,
    )

    resnet_model.add(resnet)

    resnet_model.add(Flatten())

    resnet_model.add(Dense(512, activation="relu"))

    resnet_model.add(Dropout(0.5))

    resnet_model.add(Dense(1, activation="sigmoid"))

    resnet_model.compile(loss='binary_crossentropy', 
                         optimizer=optimizers.Adam(learning_rate=0.00001),
                         metrics=[keras.metrics.AUC(), 'accuracy'])
  
    run_hist = resnet_model.fit(
      normalized_data, 
      validation_data=normalized_val_data, epochs=100, class_weight={0:w0, 1:w1}, 
      callbacks=[keras.callbacks.EarlyStopping(
                      monitor="val_auc",
                      min_delta=0,
                      patience=5,
                      verbose=1,
                      mode="auto",
                      baseline=None,
                      restore_best_weights=True,
                  )]
    )

    return resnet_model

In [ ]:
def test_model(resnet_model, test_path):
    test_data = image_dataset_from_directory(
      test_path,
      labels='inferred',
      label_mode="int",
      batch_size = 64,
      color_mode = "rgb"
    )

    normalization_layer = Rescaling(1./255)

    normalized_test_data = test_data.map(lambda x, y: (normalization_layer(x), y))
    test_image_batch, test_labels_batch = next(iter(normalized_test_data))

    results = resnet_model.evaluate(normalized_test_data)
    return results

In [ ]:
neg = 27062
pos = 4437
total = pos+neg

w0 = (1 / neg) * (total / 2.0)
w1 = (1 / pos) * (total / 2.0)

print('Peso para a classe 0: {:.2f}'.format(w0))
print('Peso para a classe 1: {:.2f}'.format(w1))

Peso para a classe 0: 0.58
Peso para a classe 1: 3.55


In [ ]:
print('Numero de imagens: NO')
!ls /content/train/no | wc -l
print('Numero de imagens: YES')
!ls /content/train/yes | wc -l

Numero de imagens: NO
0
Numero de imagens: YES
0


#### k0

In [ ]:
i = 0
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/100
329/329 [==============================] - 86s 201ms/step - loss: 0.7516 - auc: 0.6179 - accuracy: 0.5826 - val_loss: 1.0184 - val_auc: 0.4596 - val_accuracy: 0.1400
Epoch 2/100
329/329 [==============================] - 65s 196ms/step - loss: 0.6376 - auc: 0.6864 - accuracy: 0.5964 - val_loss: 1.4472 - val_auc: 0.5615 - val_accuracy: 0.1978
Epoch 3/100
329/329 [==============================] - 65s 197ms/step - loss: 0.5994 - auc: 0.7225 - accuracy: 0.6013 - val_loss: 0.5599 - val_auc: 0.7539 - val_accuracy: 0.6432
Epoch 4/100
329/329 [==============================] - 65s 196ms/step - loss: 0.5821 - auc: 0.7427 - accuracy: 0.6057 - val_loss: 0.4932 - val_auc: 0.7665 - val_accuracy: 0.7025
Epoch 5/100
329/329 [==============================] - 65s 196ms/step - loss: 0.5733 - auc: 0.7539 - accuracy: 0.6146 - val_loss: 0.5042 -

In [ ]:
clear_train_dir()

####k1

In [ ]:
i = 1
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/100
329/329 [==============================] - 73s 201ms/step - loss: 0.7205 - auc: 0.6227 - accuracy: 0.5810 - val_loss: 0.9977 - val_auc: 0.5156 - val_accuracy: 0.1400
Epoch 2/100
329/329 [==============================] - 65s 197ms/step - loss: 0.6200 - auc: 0.7062 - accuracy: 0.6143 - val_loss: 2.0468 - val_auc: 0.4699 - val_accuracy: 0.1588
Epoch 3/100
329/329 [==============================] - 65s 197ms/step - loss: 0.5962 - auc: 0.7309 - accuracy: 0.6088 - val_loss: 0.5480 - val_auc: 0.7183 - val_accuracy: 0.7173
Epoch 4/100
329/329 [==============================] - 65s 196ms/step - loss: 0.5755 - auc: 0.7515 - accuracy: 0.6171 - val_loss: 0.5041 - val_auc: 0.7434 - val_accuracy: 0.7408
Epoch 5/100
329/329 [==============================] - 65s 197ms/step - loss: 0.5726 - auc: 0.7579 - accuracy: 0.6206 - val_loss: 0.4223 -

In [ ]:
clear_train_dir()

####k2

In [ ]:
i = 2
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/100
329/329 [==============================] - 73s 199ms/step - loss: 0.7342 - auc: 0.6152 - accuracy: 0.5754 - val_loss: 0.6563 - val_auc: 0.5935 - val_accuracy: 0.7623
Epoch 2/100
329/329 [==============================] - 64s 195ms/step - loss: 0.6342 - auc: 0.6858 - accuracy: 0.5943 - val_loss: 1.6904 - val_auc: 0.5189 - val_accuracy: 0.1675
Epoch 3/100
329/329 [==============================] - 65s 196ms/step - loss: 0.6037 - auc: 0.7176 - accuracy: 0.6030 - val_loss: 0.5711 - val_auc: 0.7581 - val_accuracy: 0.6286
Epoch 4/100
329/329 [==============================] - 65s 197ms/step - loss: 0.5874 - auc: 0.7377 - accuracy: 0.6124 - val_loss: 0.5850 - val_auc: 0.7695 - val_accuracy: 0.5897
Epoch 5/100
329/329 [==============================] - 64s 195ms/step - loss: 0.5808 - auc: 0.7488 - accuracy: 0.6138 - val_loss: 0.7896 -

In [ ]:
clear_train_dir()

####k3

In [ ]:
i = 3
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/100
329/329 [==============================] - 73s 199ms/step - loss: 0.7315 - auc: 0.6250 - accuracy: 0.5810 - val_loss: 0.8598 - val_auc: 0.4809 - val_accuracy: 0.1406
Epoch 2/100
329/329 [==============================] - 64s 195ms/step - loss: 0.6358 - auc: 0.6933 - accuracy: 0.6088 - val_loss: 2.8995 - val_auc: 0.4742 - val_accuracy: 0.1415
Epoch 3/100
329/329 [==============================] - 64s 195ms/step - loss: 0.5960 - auc: 0.7300 - accuracy: 0.6154 - val_loss: 0.8747 - val_auc: 0.7597 - val_accuracy: 0.4791
Epoch 4/100
329/329 [==============================] - 64s 195ms/step - loss: 0.5770 - auc: 0.7496 - accuracy: 0.6213 - val_loss: 0.7286 - val_auc: 0.7696 - val_accuracy: 0.5361
Epoch 5/100
329/329 [==============================] - 64s 195ms/step - loss: 0.5738 - auc: 0.7548 - accuracy: 0.6228 - val_loss: 0.4985 -

In [ ]:
clear_train_dir()

####k4

In [ ]:
i = 4
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/100
329/329 [==============================] - 74s 202ms/step - loss: 0.7363 - auc: 0.6215 - accuracy: 0.5789 - val_loss: 0.4662 - val_auc: 0.6302 - val_accuracy: 0.8600
Epoch 2/100
329/329 [==============================] - 65s 197ms/step - loss: 0.6373 - auc: 0.6881 - accuracy: 0.5988 - val_loss: 1.9839 - val_auc: 0.5430 - val_accuracy: 0.1548
Epoch 3/100
329/329 [==============================] - 65s 197ms/step - loss: 0.6005 - auc: 0.7235 - accuracy: 0.6065 - val_loss: 0.7587 - val_auc: 0.7660 - val_accuracy: 0.5093
Epoch 4/100
329/329 [==============================] - 66s 199ms/step - loss: 0.5868 - auc: 0.7411 - accuracy: 0.6127 - val_loss: 0.7173 - val_auc: 0.7761 - val_accuracy: 0.5412
Epoch 5/100
329/329 [==============================] - 65s 198ms/step - loss: 0.5797 - auc: 0.7502 - accuracy: 0.6162 - val_loss: 0.5754 -

In [ ]:
clear_train_dir()

####k5

In [ ]:
i = 5
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/100
329/329 [==============================] - 75s 203ms/step - loss: 0.7552 - auc: 0.6141 - accuracy: 0.5760 - val_loss: 0.9869 - val_auc: 0.4853 - val_accuracy: 0.1400
Epoch 2/100
329/329 [==============================] - 65s 198ms/step - loss: 0.6368 - auc: 0.6860 - accuracy: 0.6001 - val_loss: 2.2894 - val_auc: 0.4885 - val_accuracy: 0.1525
Epoch 3/100
329/329 [==============================] - 65s 198ms/step - loss: 0.6020 - auc: 0.7207 - accuracy: 0.6088 - val_loss: 1.5496 - val_auc: 0.5573 - val_accuracy: 0.2500
Epoch 4/100
329/329 [==============================] - 65s 198ms/step - loss: 0.5879 - auc: 0.7358 - accuracy: 0.6080 - val_loss: 0.6369 - val_auc: 0.7710 - val_accuracy: 0.5686
Epoch 5/100
329/329 [==============================] - 65s 198ms/step - loss: 0.5786 - auc: 0.7456 - accuracy: 0.6166 - val_loss: 0.5763 -

In [ ]:
clear_train_dir()

####k6

In [ ]:
i = 6
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/100
329/329 [==============================] - 75s 203ms/step - loss: 0.7237 - auc: 0.6187 - accuracy: 0.5758 - val_loss: 0.7500 - val_auc: 0.5590 - val_accuracy: 0.3465
Epoch 2/100
329/329 [==============================] - 65s 198ms/step - loss: 0.6319 - auc: 0.6909 - accuracy: 0.5997 - val_loss: 1.6057 - val_auc: 0.5161 - val_accuracy: 0.1818
Epoch 3/100
329/329 [==============================] - 65s 198ms/step - loss: 0.5978 - auc: 0.7255 - accuracy: 0.6073 - val_loss: 0.6320 - val_auc: 0.7481 - val_accuracy: 0.5516
Epoch 4/100
329/329 [==============================] - 65s 198ms/step - loss: 0.5836 - auc: 0.7410 - accuracy: 0.6110 - val_loss: 0.5789 - val_auc: 0.7621 - val_accuracy: 0.5987
Epoch 5/100
329/329 [==============================] - 65s 198ms/step - loss: 0.5740 - auc: 0.7530 - accuracy: 0.6170 - val_loss: 0.5465 -

In [ ]:
clear_train_dir()

####k7

In [ ]:
i = 7
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/100
329/329 [==============================] - 75s 202ms/step - loss: 0.7317 - auc: 0.6188 - accuracy: 0.5743 - val_loss: 1.1119 - val_auc: 0.4569 - val_accuracy: 0.1400
Epoch 2/100
329/329 [==============================] - 65s 198ms/step - loss: 0.6345 - auc: 0.6858 - accuracy: 0.5955 - val_loss: 1.8154 - val_auc: 0.5034 - val_accuracy: 0.2024
Epoch 3/100
329/329 [==============================] - 66s 199ms/step - loss: 0.5964 - auc: 0.7301 - accuracy: 0.6095 - val_loss: 0.7111 - val_auc: 0.7632 - val_accuracy: 0.5415
Epoch 4/100
329/329 [==============================] - 65s 197ms/step - loss: 0.5850 - auc: 0.7440 - accuracy: 0.6175 - val_loss: 0.5251 - val_auc: 0.7625 - val_accuracy: 0.6520
Epoch 5/100
329/329 [==============================] - 65s 198ms/step - loss: 0.5715 - auc: 0.7594 - accuracy: 0.6233 - val_loss: 0.4710 -

In [ ]:
clear_train_dir()

####k8

In [ ]:
i = 8
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31499 files belonging to 2 classes.
Using 21000 files for training.
Found 31499 files belonging to 2 classes.
Using 10499 files for validation.
Epoch 1/100
329/329 [==============================] - 75s 203ms/step - loss: 0.7471 - auc: 0.6148 - accuracy: 0.5721 - val_loss: 0.4025 - val_auc: 0.5287 - val_accuracy: 0.8600
Epoch 2/100
329/329 [==============================] - 65s 198ms/step - loss: 0.6380 - auc: 0.6813 - accuracy: 0.5905 - val_loss: 1.4603 - val_auc: 0.5213 - val_accuracy: 0.1843
Epoch 3/100
329/329 [==============================] - 65s 198ms/step - loss: 0.6051 - auc: 0.7154 - accuracy: 0.5964 - val_loss: 0.5861 - val_auc: 0.7244 - val_accuracy: 0.6274
Epoch 4/100
329/329 [==============================] - 65s 198ms/step - loss: 0.5898 - auc: 0.7342 - accuracy: 0.6058 - val_loss: 0.5163 - val_auc: 0.7261 - val_accuracy: 0.7186
Epoch 5/100
329/329 [==============================] - 65s 198ms/step - loss: 0.5829 - auc: 0.7430 - accuracy: 0.6088 - val_loss: 0.5001 -

In [ ]:
clear_train_dir()

####k9

In [ ]:
i = 9
load_train_dir(i)
model = train_model(w0, w1)
print( test_model(model, "/content/jpg_k"+str(i)))

Found 31500 files belonging to 2 classes.
Using 21000 files for training.
Found 31500 files belonging to 2 classes.
Using 10500 files for validation.
Epoch 1/100
329/329 [==============================] - 75s 203ms/step - loss: 0.7556 - auc: 0.6141 - accuracy: 0.5775 - val_loss: 0.7084 - val_auc: 0.5489 - val_accuracy: 0.4096
Epoch 2/100
329/329 [==============================] - 65s 198ms/step - loss: 0.6395 - auc: 0.6849 - accuracy: 0.5988 - val_loss: 1.7434 - val_auc: 0.4950 - val_accuracy: 0.1530
Epoch 3/100
329/329 [==============================] - 65s 198ms/step - loss: 0.5918 - auc: 0.7332 - accuracy: 0.6124 - val_loss: 0.5276 - val_auc: 0.7274 - val_accuracy: 0.7333
Epoch 4/100
329/329 [==============================] - 66s 199ms/step - loss: 0.5742 - auc: 0.7500 - accuracy: 0.6132 - val_loss: 0.4600 - val_auc: 0.7534 - val_accuracy: 0.7917
Epoch 5/100
329/329 [==============================] - 65s 198ms/step - loss: 0.5689 - auc: 0.7579 - accuracy: 0.6171 - val_loss: 0.5084 -

In [ ]:
clear_train_dir()